# Prerequisites

### Load Packages & Settings

In [2]:
import requests, random, time, re, pandas as pd
from datetime import datetime
pd.options.display.max_rows = None

### Load Browser Agent

In [3]:
Company = None
Email = None

if Company is not None and Email is not None:
  identity = f'{Company} {Email} '
else:
  identity = ''

def Random_Agent():
  global headers_list
  user_agent_list = [
   #Chrome
    f'{identity}Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    f'{identity}Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    f'{identity}Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    f'{identity}Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    f'{identity}Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    f'{identity}Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    f'{identity}Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    f'{identity}Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
  ]
  user_agent = random.choice(user_agent_list)
  headers_list = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': user_agent
    }
  return headers_list
Random_Agent()

{'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET',
 'Access-Control-Allow-Headers': 'Content-Type',
 'Access-Control-Max-Age': '3600',
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko'}

### Load Functions

In [4]:
def make_10_digits(cik):
    cik = [cik]
    for i in range(len(cik)):
        cik[i] = str(cik[i])
        while len(cik[i]) < 10:
            cik[i] = '0' + cik[i]
    cik = cik[0]
    return cik

# Main

### Get CIK based list of Ticker

In [5]:
URL = 'https://www.sec.gov/files/company_tickers.json'
response = requests.get(URL, headers = headers_list)
data = response.json()

Ticker_output = []
for i in data:
  cik_str = data[i]['cik_str']
  ticker = data[i]['ticker']
  title = data[i]['title']
  j = pd.DataFrame([cik_str, ticker, title]).T
  Ticker_output.append(j)

Ticker_output = pd.concat(Ticker_output)
Ticker_output.columns = ['CIK', 'Ticker', 'Company']
print(len(Ticker_output))

Ticker_CIK_List = make_10_digits(list(Ticker_output['CIK']))
Ticker_HD = Ticker_output[Ticker_output.Ticker=='HD']
Ticker_HD.head()

10661


,CIK,Ticker,Company
0,354950,HD,"HOME DEPOT, INC."


### SEC EDGAR Function

In [6]:
def SEC_EDGAR(cik_input):
    # Get Latest 10K/Q Filing
    def Extract_Filing(cik_input):
        global accessionNumber, filingDate, reportDate, acceptanceDateTime, act, form, fileNumber, filmNumber, items, size, isXBRL, isInlineXBRL, primaryDocument, primaryDocDescription, df_Extract_Latest_Filings
        cik = make_10_digits(cik_input)
        URL = f'https://data.sec.gov/submissions/CIK{cik}.json'
        response = requests.get(URL, headers = headers_list)
        data = response.json()
        try:
            time.sleep(1)
            accessionNumber = data['filings']['recent']['accessionNumber']
            filingDate = data['filings']['recent']['filingDate']
            reportDate = data['filings']['recent']['reportDate']
            acceptanceDateTime = data['filings']['recent']['acceptanceDateTime']
            act = data['filings']['recent']['act']
            form = data['filings']['recent']['form']
            fileNumber = data['filings']['recent']['fileNumber']
            filmNumber = data['filings']['recent']['filmNumber']
            items = data['filings']['recent']['items']
            size = data['filings']['recent']['size']
            isXBRL = data['filings']['recent']['isXBRL']
            isInlineXBRL = data['filings']['recent']['isInlineXBRL']
            primaryDocument = data['filings']['recent']['primaryDocument']
            primaryDocDescription = data['filings']['recent']['primaryDocDescription']
        except:
            pass
        Extract_Filing = pd.DataFrame([accessionNumber, filingDate, reportDate, acceptanceDateTime, act, form, fileNumber, filmNumber, items, size, isXBRL, isInlineXBRL, primaryDocument, primaryDocDescription]).T
        Extract_Filing.columns = ['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
        Extract_Filing = Extract_Filing[Extract_Filing['form'].isin(['FORM 10-Q', '10-K', '10-Q'])].sort_values('reportDate', ascending = False).reset_index().drop('index', axis=1)
        Extract_Filing['cik'] = cik
        return Extract_Filing

    Extract_Latest_Filings = []
    Ticker_CIK_List = make_10_digits(cik_input)
    Extract_Latest_Filings.append(Extract_Filing(Ticker_CIK_List))   
    df_Extract_Latest_Filings = pd.concat(Extract_Latest_Filings).drop_duplicates()
    print(len(df_Extract_Latest_Filings))

    df_Extract_Latest_Filings = df_Extract_Latest_Filings[df_Extract_Latest_Filings['filingDate']>=f'{datetime.now().year-2}-01-01']

    # Define extract methods for text and data
    def Extract_Data(raw_10k):
        df = []
        for i in range(len(re.findall(fr'defref_us-gaap_(.*?)</td>\n(.*?)<span></span>', raw_10k))):
            j = re.findall(fr'defref_us-gaap_(.*?)</td>\n(.*?)<span></span>', raw_10k)[i]
            gaap_code = "defref_us-gaap_" + j[0].split("\', window );")[0]
            gaap_name = j[0].split("""\', window );">""")[1].split('</a>')[0].replace('<strong>','').replace(':</strong>','').replace('</strong>','').replace('#8217;','')
            gaap_value = j[1].replace('<td class="nump">','').replace('$ ', '').replace('<td class="text">&#160;','').replace('<td class="num">','').replace('<td class="text">','')

            k = pd.DataFrame([gaap_code, gaap_name, gaap_value]).T.rename(columns={0:'gaap_code', 1:'gaap_name', 2:'gaap_value'})
            df.append(k)
        df = pd.concat(df)

        def filter_rows(df):
            return df[~df['gaap_code'].str.endswith('TextBlock') & ~df['gaap_code'].str.endswith('Abstract') & ~df['gaap_code'].str.endswith('Member') & ~df['gaap_code'].str.endswith('defref_us-gaap_FiscalPeriod')]
        df = filter_rows(df)
        return df

    def Extract_Text(raw_10k):
        df = []
        for i in range(len(re.findall(fr'defref_dei_(.*?)</td>\n(.*?)<span></span>', raw_10k))):
            j = re.findall(fr'defref_dei_(.*?)</td>\n(.*?)<span></span>', raw_10k)[i]
            gaap_code = "defref_dei_" + j[0].split("\', window );")[0]
            gaap_name = j[0].split("""\', window );">""")[1].split('</a>')[0].replace('<strong>','').replace(':</strong>','').replace('</strong>','').replace('#8217;','')
            gaap_value = j[1].replace('<td class="text">','').replace('$ ', '').replace('<td class="text">&#160;','').replace('<td class="num">','').replace('<td class="text">','').replace('<td class="nump">', '').replace('&#160;', ' ')

            k = pd.DataFrame([gaap_code, gaap_name, gaap_value]).T.rename(columns={0:'gaap_code', 1:'gaap_name', 2:'gaap_value'})
            df.append(k)
        df = pd.concat(df)

        def filter_rows(df):
            return df[~df['gaap_code'].str.endswith('TextBlock') & ~df['gaap_code'].str.endswith('Abstract') & ~df['gaap_code'].str.endswith('Member') & ~df['gaap_code'].str.endswith('defref_us-gaap_FiscalPeriod') & ~df['gaap_code'].str.endswith('Authorization')]
        df = filter_rows(df)
        df = df[df['gaap_value']!='&#160;']
        return df


    # Extract all data from most recent SEC filing (10K/Q)
    def f_GAAP_Output(record):
        cik = df_Extract_Latest_Filings['cik'].iloc[record]
        accessionNumber = df_Extract_Latest_Filings['accessionNumber'].iloc[record]
        primaryDocument = df_Extract_Latest_Filings['primaryDocument'].iloc[record]
        r = requests.get(f'https://www.sec.gov/Archives/edgar/data/{cik}/{accessionNumber.replace("-", "")}/{accessionNumber}.txt', headers=headers_list)
        raw_10k = r.text
        GAAP_Output_text = Extract_Text(raw_10k)
        GAAP_Output_data = Extract_Data(raw_10k)
        GAAP_Output = pd.concat([GAAP_Output_text, GAAP_Output_data])
        GAAP_Output = GAAP_Output.reset_index().drop('index', axis=1)
        return GAAP_Output

    # Show results
    return f_GAAP_Output(0)

### Output

In [7]:
# Example (Home Depot)
SEC_EDGAR(Ticker_HD['CIK'][0])

39


,gaap_code,gaap_name,gaap_value
0,defref_dei_DocumentType,Document Type,10-Q
1,defref_dei_DocumentQuarterlyReport,Document Quarterly Report,true
2,defref_dei_DocumentPeriodEndDate,Document Period End Date,"Oct. 29, 2023"
3,defref_dei_DocumentTransitionReport,Document Transition Report,false
4,defref_dei_EntityFileNumber,Entity File Number,1-8207
5,defref_dei_EntityRegistrantName,Entity Registrant Name,"HOME DEPOT, INC."
6,defref_dei_EntityIncorporationStateCountryCode,"Entity Incorporation, State or Country Code",DE
7,defref_dei_EntityTaxIdentificationNumber,Entity Tax Identification Number,95-3261426
8,defref_dei_EntityAddressAddressLine1,"Entity Address, Address Line One",2455 Paces Ferry Road
9,defref_dei_EntityAddressCityOrTown,"Entity Address, City or Town","Atlanta,"
